### Conjugate gradient method

**Problem of interest**

Given an $n$-by-$n$ symmetric positive definite (SPD) matrix $A$ and a $\mathbb{R}^n$-vector $b$, find $\mathbb{R}^n$-vector $x$ such that

$$ Ax = b. $$


#### Method: Conjugate gradient method

**Data**

- $x_0 \in \mathbb{R}^n$: initial guess

**Initialize**

- $d_0=r_0=b-Ax_0$

**Main computation**

- **for** k = 0, 1, 2, ..., $n$ - 1
    - **if** $r_k = 0$ **stop**, **end**
    - $\alpha_{k}=\frac{r_{k}^{T} r_{k}}{d_{k}^{T} A d_{k}}$ (compute $d_k$ component of error)
    - $x_{k+1}=x_{k}+\alpha_{k} d_{k}$ (subtract it out)
    - $r_{k+1}=r_{k}-\alpha_{k} A d_{k}$ (compute the new residual)
    - $\beta_{k}=\frac{r_{k+1}^{T} r_{k+1}}{r_{k}^{T} r_{k}}$ (compute $d_k$ component of the residual)
    - $d_{k+1}=r_{k+1}+\beta_{k} d_{k}$ (conduct Gram-Schmidt with respect to $A$-inner product)
**end**

**History**

1. First proposed by Schmidt (1908) [^1] (*the* Schmidt in Gram-Schmidt)
1. Independently re-invented by Fox, Huskey, and Wilkinson (1948) [^2]
1. Hestenes and Stiefel (1952) made this idea explicit and practical. [^3]
1. CG does not reach the solution in $n$ steps in practice due to round off errors. It became popular only after Reid (1971) showed its value as an iterative method for large, sparse matrices. [^4] 

[^1]: Schmidt (1908) Uber die Auflosung linearer Gleichungen mit Unendlich vielen unbekannten (accent removed)

[^2]: Fox, Huskey, and Wilkinson (1948) Notes on the solution of algebraic linear simultaneous equations

[^3]: Hestenes and Stiefel (1952) Methods of conjugate gradients for solving linear systems 

[^4]: Reid (1971) On the method of conjugate gradients for the solution of large
	sparse systems of linear equations

#### Notation/Settings

| expression | meaning |
|---|---|
| $x$ | true solution ($Ax=b$) |
| $x_k$ | $k$-th approximate solution by the conjugate gradient method ($k=0,1,2,\cdots$)|
| $e_k$ | $=x - x_k$ (error caused by $x_k$) |
| $r_k$ | $=b-Ax_k$ (residual caused by $x_k$) |
| $d_k$ | conjugate directions |
| $(u,v)$ | $=v^T u$ (standard inner product) |
| $(u,v)_A$ | $=v^T A u$ (A-inner product, where $A$ is a SPD matrix) |
| $v \perp_A u$ | $(u,v)_A =0$ (A-orthogonality) |

#### Idea behind the conjugate gradient method



##### Warm up (Gaussian elimination)

1. Change the perspective

View the process of finding the solution as removing components of error one by one. Unless we are extremely lucky, our error caused by the initial guess will be full of possible components. 

For the moment, to avoid introducing too many symbols, let us override the notations and let $d_k$'s be canonical basis of $\mathbb{R}^{n}$. 

We can expand the initial error in the canonical basis.

$$
e_{0}=\sum_{k=0}^{n-1} \eta_k d_k
$$



2. Remove component one by one

Then the back substitution step can be seen as removing $d_{n-2}$ component from the error, then $d_{n-3}$ component, all the way to $d_{0}$. ($d_{n-1}$ component is already absent since the last component of $x$ is precise.)

$$
\begin{bmatrix}
1&-\frac{1}{2}&\frac 3 4 & \frac 7 4 \\
0&1& \frac 3 5 & \frac {26} 5\\
0&0&1& \underbrace{2}_{x_1} 
\end{bmatrix}
\longrightarrow	
\begin{bmatrix}
1&-\frac{1}{2}&0 & -1 \\
0&1& 0& 4\\
0&0&1& \underbrace{2}_{x_2}  
\end{bmatrix}
\longrightarrow	
\begin{bmatrix}
1&0&0 & 1 \\
0&1& 0& 4\\
0&0&1& \underbrace{2}_{x_3}  
\end{bmatrix}
$$

![Gaussian elimination from error improvement point of view](../images/CG01.png)

**Remark**


- Removing $d_k$ from the approximate solution $x_k$ and removing it from the error $e_k$ are equivalent because they differ only by a fixed vector, the true solution $x$, that is, $e_k = x - x_k$. But the signs of $e_k$ and $x_k$ are opposite.




3. Algorithm-friendly summary


Given $x_k$, hence $e_k = x - x_k$, take exact step to remove $d_k$ component each time

$$
e_{k+1}=e_k - \eta_k d_k.
$$

or equivalently, 

$$
x_{k+1}=x_k + \eta_k d_k,
$$

The exact component $\eta_k$ in $d_k$ direction can be computed by the condition $e_{k+1} \perp d_k$ since we just subtracted out $d_k$:

$$
\begin{split}
e_{1}&=\sum_{j=1}^{n-1} \eta_k d_j
\\
e_{2}&=\sum_{j=2}^{n-1} \eta_k d_j
\\
&\vdots
\\
e_{k+1}&=\sum_{j=k+1}^{n-1} \eta_k d_j.
\end{split}
$$

By taking dot product with $d_k$ on both sides of the error update rule $e_{k+1}=e_k - \eta_k d_k$, we can find

$$
\eta_k = \frac{d_k^T e_{k}}{d_k^T d_k}.
$$


**Observation**

- $e_k$ is not computable since we do not know the true solution $x$.
- The residual $r_k:=b - Ax_k$ is computable. 
- Since $r_k = b - A x_k = A x - A x_k = A(x - x_k) = Ae_k$, the residual and the error are related:

$$
r_k = Ae_k
$$

- Though not at all trivial, these observations suggest a possibility to play between A-orthogonality and orthogonality for our needs.

##### Devising conjugate gradient method

**Notation**

From now on, $\{d_k\}$ is an A-orthogonal basis.

1. View finding the solution as removing $d_k$ component at a time. 

2. Suppose we have an A-orthogonal basis $\{d_k\}_{k=0}^{n-1}$. (We will discuss how to obtain it. For now, assume it is possible.) We can expand the initial error in this basis. Again, it is likely that the initial guess contains all components of $d_k$'s:

$$
e_{0}=\sum_{k=0}^{n-1} \eta_k d_k
$$

3. Suppose we can remove $d_k$ component from the error each time (this can be achieved by finding $\eta_k$ momentarily):

$$
e_{k+1}=e_k - \eta_k d_k,
$$

or equivalently from approximate solutions

$$
x_{k+1}=x_k - \eta_k d_k.
$$


Then, we are left with less and less $d_k$ components in the error:

$$
\begin{split}
e_{1}&=\sum_{j=1}^{n-1} \eta_k d_j
\\
e_{2}&=\sum_{j=2}^{n-1} \eta_k d_j
\\
&\vdots
\\
e_{k+1}&=\sum_{j=k+1}^{n-1} \eta_k d_j.
\end{split}
$$

Thus, since $e_{k+1}$ has no $d_k$ component (it has just been removed), we must have $e_{k+1} \perp_{A} d_k$. Use this fact to the error update rule, then we can compute $\eta_k$:

$$
\eta_k = \frac{d_k^T A e_k}{d_k^T A d_k}=\frac{d_k^T r_k}{d_k^T A d_k} \quad \text{(computable)}.
$$

This is where $A$-orthogonality is used in a nice way.

In the algorithm, we use $\eta_k$ formula to update $x_k$'s since $e_k$ is not computable during implementation.

4. New residuals guarantee new direction to explore to remove further $d$-components. In fact, it is orthogonal to all the past directions $d_k$'s:

$$ 
r_{k+1} \perp \mathrm{span} \{ d_j \ : \  0 \le j \le k \}
$$

since, by mutual A-orthogonality between $d$'s, 

$$
d_j^T r_{k+1} = d_j^T A e_{k+1} = \sum_{i=k+1}^{n-1} \eta_k d_j^T A  d_i = 0.
$$

